<a href="https://colab.research.google.com/github/annvorosh/GB/blob/NLPmed/NLPmed_L01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import files

# Загрузка файла
uploaded = files.upload()

# Вывод информации о загруженных файлах
for filename in uploaded.keys():
    print(f'Файл {filename} был загружен, содержимое: {len(uploaded[filename])} символов')


Saving dates.txt to dates (1).txt
Файл dates (1).txt был загружен, содержимое: 63878 символов


In [23]:
import re

# Чтение файла
with open('dates.txt', 'r') as file:
    content = file.readlines()

In [24]:
# Словарь для преобразования сокращенных названий месяцев в их числовые значения
month_dict = {
    'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06',
    'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
}

# Регулярное выражение для извлечения дат
date_pattern = (
    r'(\d{1,2}[.-/]\d{1,2}[.-/]\d{2,4})'  # MM.DD.YY or MM.DD.YYYY or ... / -
    r'|'
    r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{1,2},?\s\d{2,4}'  # Mon DD, YYYY
    r'|'
    r'\d{1,2}\s(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{2,4}'  # DD Mon YYYY
    r'|'
    r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{2,4}'  # Mon YYYY
    r'|'
    r'\d{1,2}[/.]\d{2,4}'  # MM/YYYY
    r'|'
    r'\d{4}'  # YYYY (only year)
    r'|'
    r'\b\d{1,2}[.-/]\d{1,2}[.-/]\d{2,4}\b'  # MM.DD.YY or MM.DD.YYYY or ... / -
    r'|'
    r'\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{1,2},?\s\d{2,4}\b'  # Mon DD, YYYY
    r'|'
    r'\b\d{1,2}\s(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{2,4}\b'  # DD Mon YYYY
    r'|'
    r'\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{2,4}\b'  # Mon YYYY
    r'|'
    r'\b\d{1,2}[/.]\d{2,4}\b'  # MM/YYYY
    r'|'
    r'\b\d{4}\b'  # YYYY (only year)
    r'|'
    r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{4}'  # Mon YYYY without day
    r'|'
    r'\d{1,2}/\d{4}'  # MM/YYYY without day
    r'|'
    r'\d{1,2}\s(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\b\s\d{2,4}'  # DD Mon YYYY without comma
)

In [25]:
# Извлечение и нормализация дат

normalized_dates = []

for index, line in enumerate(content):
    # Извлечение дат из строки
    matches = re.finditer(date_pattern, line)

    found_dates = []  # Список для хранения обнаруженных дат в текущей строке

    for match in matches:
        date = match.group(0)  # Извлекаем всю найденную дату
        parts = re.split(r'\s+|\.|/', date)  # Разделение даты на части

        if len(parts) == 3:  # Если дата содержит три части: месяц, день, год
            month, day, year = parts

            # Нормализация месяца и дня
            if not month.isdigit():
                month = month_dict[month[:3]]  # Получение числового значения месяца из словаря
            if len(day) == 1:
                day = '0' + day
            if len(month) == 1:
                month = '0' + month
            if len(year) == 2:
                if int(year) > 24:
                    year = '19' + year
                else:
                    year = '20' + year
            normalized_date = f"{year}-{month}-{day}"  # Форматированная дата
            found_dates.append(normalized_date)
            break  # Прерываем цикл после нахождения первой даты

        elif len(parts) == 2:  # Если дата содержит две части: месяц, год
            month, year = parts
            if not month.isdigit():
                month = month_dict[month[:3]]  # Получение числового значения месяца из словаря
            if len(month) == 1:
                month = '0' + month
            if len(year) == 2:
                if int(year) > 24:
                    year = '19' + year
                else:
                    year = '20' + year
            day = '01'  # Добавляем значение "01" для дня
            normalized_date = f"{year}-{month}-{day}"  # Форматированная дата
            found_dates.append(normalized_date)
            break  # Прерываем цикл после нахождения первой даты

        elif len(parts) == 1:  # Если дата состоит только из года
            year = parts[0]
            if len(year) == 2:
                if int(year) > 24:
                    year = '19' + year
                else:
                    year = '20' + year
            month = '01'  # Добавляем значение "01" для месяца
            day = '01'  # Добавляем значение "01" для дня
            normalized_date = f"{year}-{month}-{day}"  # Форматированная дата
            found_dates.append(normalized_date)

    # Если в строке не было найдено дат, добавляем пустую дату в список
    if not found_dates:
        found_dates.append('-')

    # Добавляем индекс строки и первую найденную дату в общий список
    normalized_dates.append((index, found_dates[0]))

# Запись результатов в файл
with open('assignment_VoroshilovaA.txt', 'w') as output_file:
    for index, normalized_date in normalized_dates:
        if normalized_date:
            output_file.write(f'{index}\t{normalized_date}\n')
        else:
            output_file.write(f'{index}\t\n')

# Вывод результатов
for index, normalized_date in normalized_dates:
    if normalized_date:
        print(f'{index}\t\t{normalized_date}')
    else:
        print(f'{index}\t\t-')


0		1999-07-21
1		1981-01-01
2		1980-03-01
3		1990-07-11
4		1986-06-02
5		2012-03-01
6		1985-06-18
7		2007-09-01
8		1975-12-01
9		1986-08-01
10		1979-01-01
11		1972-09-17
12		2003-06-Oct
13		1994-11-24
14		1987-01-01
15		1992-08-16
16		1997-12-12
17		1976-09-30
18		2010-26-May
19		1981-01-01
20		1973-01-01
21		1997-12-08
22		1994-09-13
23		1984-09-01
24		1986-04-18
25		1977-02-14
26		1997-07-13
27		2007-04-01
28		1988-04-12
29		1971-06-10
30		1993-12-12
31		2002-01-01
32		1981-03-26
33		2010-08-01
34		2010-08-01
35		1979-10-06
36		1977-08-31
37		1975-01-01
38		1994-09-09
39		1975-12-01
40		1979-09-21
41		1980-03-18
42		1983-01-01
43		1977-08-30
44		1972-04-01
45		1995-05-12
46		1990-06-08
47		2008-02-09
48		1977-08-21
49		1998-01-27
50		1982-05-02
51		1979-02-18
52		1983-01-01
53		1979-01-01
54		1987-10-11
55		1980-10-17
56		1983-10-Feb
57		2013-05-18
58		1980-01-02
59		2011-08-24
60		1979-01-29
61		1977-05-21
62		1979-02-12
63		1978-09-08
64		1981-07-09
65		2013-01-01
66		2007-01-01
67

In [28]:
# Подсчет количества дат и строк без дат
total_dates = len(normalized_dates)
dates_not_found = sum(1 for _, date in normalized_dates if date == '-')

# Вычисление процента дат, не обнаруженных в тексте
percent_dates_not_found = (dates_not_found / total_dates) * 100

# Вывод результатов
print(f'Всего дат: {total_dates}')
print(f'Даты, не обнаруженные в тексте: {dates_not_found}')
print(f'Процент дат, не обнаруженных в тексте: {percent_dates_not_found:.2f}%')

# Подсчет количества дат, где месяц указан буквами
dates_with_month_name = sum(1 for _, date in normalized_dates if len(re.findall(r'[a-zA-Z]{3}', date)) > 0)

# Вывод результатов
print(f'Количество дат, где месяц указан буквами: {dates_with_month_name}')

Всего дат: 500
Даты, не обнаруженные в тексте: 6
Процент дат, не обнаруженных в тексте: 1.20%
Количество дат, где месяц указан буквами: 19
